In [9]:
# Lets first import the libraries we will need 
from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Select
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models.widgets import Tabs, Panel
import pandas as pd
import numpy as np

In [10]:
# Second step will be importing the data to our Jupyter Notebook
# The data I used to build this project can be found here:
# https://www.theguardian.com/sport/datablog/2012/jun/25/olympic-medal-winner-list-data#data

filepath = '#'

olympics = pd.read_csv(filepath, 
    skiprows = [0,1,2], header = 1)

In [11]:
# After the first look at the database I decided to use the columns MEDAL, NOC, and EDITION
# To build graphics using Bokeh server

data = olympics.loc[:, ['Medal', 'NOC', 'Edition', 'Gender']]
data.sort_values(by = ['NOC', 'Edition'], inplace = True)

In [12]:
# The following step was creating these functions to count how many GOLD, SILVER, and BRONZE medals 
# Each country got over the years (from 1896 to 2008)

def separete_gold(value):
    if value == 'Gold':
        return 1
    else: 
        return 0
    
def separete_silver(value):
    if value == 'Silver':
        return 1
    else: 
        return 0

def separete_bronze(value):
    if value == 'Bronze':
        return 1
    else: 
        return 0
    
def separete_medals_men(value):
    if value == 'Men':
        return 1
    else:
        return 0
    
def separete_medals_women(value):
    if value == 'Women':
        return 1
    else: 
        return 0
# Adding new columns to our database

data['Gold'] = data.Medal.apply(separete_gold)
data['Silver'] = data.Medal.apply(separete_silver)
data['Bronze'] = data.Medal.apply(separete_bronze)
data['Man'] = data.Gender.apply(separete_medals_men)
data['Woman'] = data.Gender.apply(separete_medals_women)

In [13]:
# Here we rearange the data so that we can see how the medals are distributed over 
# the years. We make sure that every NOC will be represented every 4 years and
# we get the correct corresponding YEAR - NOC - GOLD, SILVER, BRONZE

lista_anos = list(data['Edition'].unique())
lista_noc = list(data['NOC'].unique())
lista_plot = []

lista_noc_anos = []
for noc in lista_noc:
    for year in lista_anos:
        lista_aux = list(data[(data['NOC'] == noc) 
                & (data['Edition'] == year)][['Gold', 'Silver', 'Bronze', 'Man', 'Woman']].sum().values)
        lista_aux.extend([year, noc])
        lista_noc_anos.append(lista_aux)

In [14]:
# Finally we make a new DataFrame using the data we reorganized
data_strutured = np.array(lista_noc_anos)
dataframe = pd.DataFrame(data_strutured, columns=['Gold',
                                                 'Silver',
                                                 'Bronze',
                                                 'Man',
                                                 'Woman',
                                                 'Year',
                                                 'NOC'])


dataframe.Gold = np.array(dataframe.Gold, dtype=np.int32)
dataframe.Silver = np.array(dataframe.Silver, dtype=np.int32)
dataframe.Bronze = np.array(dataframe.Bronze, dtype=np.int32)
dataframe.Man = np.array(dataframe.Man, dtype=np.int32)
dataframe.Woman = np.array(dataframe.Woman, dtype=np.int32)
dataframe.Year = np.array(dataframe.Year, dtype=np.int32)

In [15]:
# Here we start to prepare our data to be used to draw the graphics
source_gold = ColumnDataSource(data = {
    'x': dataframe[dataframe['NOC'] == 'USA'].sort_values(by='Year')['Year'],
    'y': dataframe[dataframe['NOC'] == 'USA'].sort_values(by='Year')['Silver'],
})

source_silver = ColumnDataSource(data = {
    'x': dataframe[dataframe['NOC'] == 'USA'].sort_values(by='Year')['Year'],
    'y': dataframe[dataframe['NOC'] == 'USA'].sort_values(by='Year')['Gold'],
})

source_bronze = ColumnDataSource(data = {
    'x': dataframe[dataframe['NOC'] == 'USA'].sort_values(by='Year')['Year'],
    'y': dataframe[dataframe['NOC'] == 'USA'].sort_values(by='Year')['Bronze'],
})

source_man = ColumnDataSource(data = {
    'x': dataframe[dataframe['NOC'] == 'USA'].sort_values(by='Year')['Year'],
    'y': dataframe[dataframe['NOC'] == 'USA'].sort_values(by='Year')['Man'],
})

source_woman = ColumnDataSource(data = {
    'x': dataframe[dataframe['NOC'] == 'USA'].sort_values(by='Year')['Year'],
    'y': dataframe[dataframe['NOC'] == 'USA'].sort_values(by='Year')['Woman'],
})

# Set the limits for x_axis
xmin, xmax = min(dataframe['Year']), max(dataframe['Year'])

In [16]:
# Creating a figure and writing a callback
plot = figure(x_axis_label='Year', 
              y_axis_label = 'Total Medals by Types',
             x_range= (xmin - 8, xmax + 8))

plot_gender = figure(x_axis_label='Year', 
              y_axis_label = 'Total Medals by Gender',
             x_range= (xmin - 8, xmax + 8))

def update_medals(attr, old, new):
    noc = menu_noc_medals.value
    new_data_gold ={
        'x': dataframe[dataframe['NOC'] == noc].sort_values(by='Year')['Year'],
        'y': dataframe[dataframe['NOC'] == noc].sort_values(by='Year')['Gold'],
    }
    new_data_silver ={
        'x': dataframe[dataframe['NOC'] == noc].sort_values(by='Year')['Year'],
        'y': dataframe[dataframe['NOC'] == noc].sort_values(by='Year')['Silver'],
    }
    new_data_bronze ={
        'x': dataframe[dataframe['NOC'] == noc].sort_values(by='Year')['Year'],
        'y': dataframe[dataframe['NOC'] == noc].sort_values(by='Year')['Bronze'],
    }
    source_gold.data = new_data_gold
    source_silver.data = new_data_silver
    source_bronze.data = new_data_bronze
    
def update_gender(attr, old, new):
    noc = menu_noc_gender.value
    new_data_man ={
        'x': dataframe[dataframe['NOC'] == noc].sort_values(by='Year')['Year'],
        'y': dataframe[dataframe['NOC'] == noc].sort_values(by='Year')['Man'],
    }
    new_data_woman ={
        'x': dataframe[dataframe['NOC'] == noc].sort_values(by='Year')['Year'],
        'y': dataframe[dataframe['NOC'] == noc].sort_values(by='Year')['Woman'],
    }
    source_man.data = new_data_man
    source_woman.data = new_data_woman
    

In [17]:
# Creating menus and adding on_change methods
menu_noc_medals = Select(options=lista_noc,
value = 'USA',title ='Countries')

menu_noc_gender = Select(options=lista_noc,
value = 'USA',title ='Countries')

menu_noc_medals.on_change('value', update_medals)

menu_noc_gender.on_change('value', update_gender)

In [18]:
# Finally plotting and organazing everything in a layout
plot.line(x = 'x', y = 'y', source = source_gold,line_color ='gold', legend='gold')
plot.circle(x = 'x', y = 'y', source = source_gold,color ='gold', legend='gold')
plot.line(x = 'x', y = 'y', source = source_silver, line_color ='silver', legend='silver')
plot.square(x = 'x', y = 'y', source = source_silver, line_color ='silver', fill_color =None, legend='silver')
plot.line(x = 'x', y = 'y', source = source_bronze, line_color ='brown', legend='bronze')
plot.triangle(x = 'x', y = 'y', source = source_bronze, line_color ='brown', fill_color=None, legend='bronze')
plot.legend.location = 'top_left'

plot_gender.line(x = 'x', y = 'y', source = source_man,line_color ='black', legend='men')
plot_gender.asterisk(x = 'x', y = 'y', source = source_man,line_color ='black', fill_color=None, legend='men')
plot_gender.line(x = 'x', y = 'y', source = source_woman,line_color ='blue', legend='women')
plot_gender.diamond(x = 'x', y = 'y', source = source_woman,line_color ='blue', fill_color = None, legend='women')
plot_gender.legend.location = 'top_center'

first = Panel(child = column(menu_noc_medals, plot), title = 'BY KIND')
second = Panel(child = column(menu_noc_gender, plot_gender), title = 'BY GENDER')

tabs = Tabs(tabs=[first, second])

# layout = column(menu_noc_medals, plot)

curdoc().add_root(tabs)